In [2]:
from FuncionAzul import *

# Prueba 

In [3]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/23 22:40:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/23 22:40:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/23 22:40:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/23 22:40:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 22:40:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/23 2

...

22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:22:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 0

22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 0

22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 0

22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 0

22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 00:23:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/24 0

In [ ]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = True)